In [1]:
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np
from scipy.stats import uniform,norm,t,binom,expon,chi2,gamma,poisson,beta,dirichlet
from scipy.stats import multivariate_normal as mvnorm
from sklearn.linear_model import LinearRegression as Linear
from scipy.optimize import brentq,minimize
from scipy.special import perm,comb
from time import time
import pandas as pd
import qmcpy as qp

import warnings
warnings.filterwarnings("ignore")

**17.1**

In [2]:
def bootstrap(S,B):
    parS=np.mean(S)
    stdS=np.sqrt(np.sum((S-parS)**2)/(S.size-1))
    
    parsBS=np.zeros(B)
    stdsBS=np.zeros(B)
    for i in range(B):
        BS=np.random.choice(S,S.size)
        parsBS[i]=np.mean(BS)
        stdsBS[i]=np.sqrt(np.sum((BS-parsBS[i])**2)/(S.size-1))
        
    parsBS2=stdS/stdsBS*(parsBS-parS)+parS
    alpha=0.05
    itv1=(2*parS-np.quantile(parsBS,1-alpha/2),\
          2*parS-np.quantile(parsBS,alpha/2))
    itv2=(2*parS-np.quantile(parsBS2,1-alpha/2),\
          2*parS-np.quantile(parsBS2,alpha/2))
    
    return itv1,itv2

def experiment(par,rvs,n=20,B=20,N=100):
    count1=0
    count2=0
    count3=0
    length1=0
    length2=0
    for t in range(N):
        S=rvs(n)
        itv1,itv2=bootstrap(S,B)
        count1+=par>=itv1[0] and par<=itv1[1]
        count2+=par>=itv2[0] and par<=itv2[1]
        count3+=itv2[1]-itv2[0]>itv1[1]-itv1[0]
        length1+=itv1[1]-itv1[0]
        length2+=itv2[1]-itv2[0]
    
    print('coverage 1: {}'.format(count1/N))
    print('coverage 2: {}'.format(count2/N))
    print('longer: {}'.format(count3/N))
    print('length 1: {:.3f}'.format(length1/N))
    print('length 2: {:.3f}'.format(length2/N))

In [3]:
def rvs(size):
    S=np.zeros(size)
    for i in range(size):
        S[i]=norm.rvs(size=16).mean()
        
    return S

# print('n=10:')
# experiment(0,lambda size: rvs(size=size),n=10)
# print('n=20:')
# experiment(0,lambda size: rvs(size=size),n=20)
# print('n=50:')
# experiment(0,lambda size: rvs(size=size),n=50)
# print('n=200:')
# experiment(0,lambda size: rvs(size=size),n=200)

In [4]:
def rvs(size):
    S=np.zeros(size)
    for i in range(size):
        S[i]=qp.Gaussian(qp.Halton(1,randomize='QRNG')).gen_samples(16).mean()
        
    return S

# print('QRNG:')
# experiment(0,rvs)

def rvs(size):
    S=np.zeros(size)
    for i in range(size):
        S[i]=qp.Gaussian(qp.Halton(1,randomize='OWEN')).gen_samples(16).mean()
        
    return S

# print('OWEN:')
# experiment(0,rvs)

In [5]:
def rvs(size):
    S=np.zeros(size)
    for i in range(size):
        S[i]=qp.Gaussian(qp.Lattice(1)).gen_samples(16).mean()
        
    return S

# experiment(0,rvs)

In [6]:
def rvs(size):
    S=np.zeros(size)
    for i in range(size):
        S[i]=qp.Gaussian(qp.Sobol(1,randomize='LMS')).gen_samples(16).mean()
        
    return S

print('LMS:')
experiment(0,rvs)

def rvs(size):
    S=np.zeros(size)
    for i in range(size):
        S[i]=qp.Gaussian(qp.Sobol(1,randomize='DS')).gen_samples(16).mean()
        
    return S

print('DS:')
experiment(0,rvs)

LMS:
coverage 1: 0.0
coverage 2: 0.0
longer: 0.0
length 1: nan
length 2: nan
DS:
coverage 1: 0.78
coverage 2: 0.83
longer: 0.91
length 1: 0.071
length 2: 0.079
